In [5]:
#firstly, extracting all items inside a user's iventory (only cs2 items)
import requests
import time
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                  "AppleWebKit/537.36 (KHTML, like Gecko) "
                  "Chrome/117.0.0.0 Safari/537.36"
}
def fetch_inventory_page(steamid64: str, appid: int, contextid: int, start: str = None):
    url = f"https://steamcommunity.com/inventory/{steamid64}/{appid}/{contextid}"
    resp = requests.get(url,headers=HEADERS, timeout=15)
    resp.raise_for_status()
    return resp.json()
def inventory_items_for_user(steamid64: str, appid: int, contextid: int, delay_between_requests: float = 0.5):
    items = []
    seen_assetids = set()
    start = None

    while True:
        data = fetch_inventory_page(steamid64, appid, contextid, start)
        # check if inventory is private or empty
        if not data or "assets" not in data or "descriptions" not in data:
            # Might be private or zero items; return whatever we have or raise
            return items, data

        assets = data.get("assets", [])
        descriptions = data.get("descriptions", [])

        # Build lookup for descriptions by (classid, instanceid)
        desc_lookup = {}
        for d in descriptions:
            key = (str(d.get("classid")), str(d.get("instanceid", "0")))
            desc_lookup[key] = d

        for a in assets:
            assetid = a.get("assetid")  # unique per item
            if assetid in seen_assetids:
                continue
            seen_assetids.add(assetid)

            classid = str(a.get("classid"))
            instanceid = str(a.get("instanceid", "0"))
            desc = desc_lookup.get((classid, instanceid)) or {}

            item = {
                "assetid": assetid,
                "classid": classid,
                "instanceid": instanceid,
                "amount": a.get("amount", 1),
                "market_tradable_restriction": a.get("market_tradable_restriction"),
                # descriptive fields:
                "name": desc.get("name"),
                "type": desc.get("type"),
                "market_hash_name": desc.get("market_hash_name"),
                "tradable": desc.get("tradable"),
                "tags": desc.get("tags", []),
                "marketable": desc.get("marketable"),
                # entire desc for debugging
                "description_raw": desc
            }
            if item["tradable"] ==1 and item["marketable"] == 1:
                items.append(item)

        # pagination
        more = data.get("more_items") or False
        # some responses give 'last_assetid' to use as start; others use 'more_start'
        if more:
            start = data.get("last_assetid") or data.get("more_start")
            if not start:
                # fallback: if server indicates 'more_items' but no token, stop to avoid infinite loop
                break
            time.sleep(delay_between_requests)  # be polite
        else:
            break

    return items, data

In [6]:
steamid64 = "76561198923974658"   # example SteamID64
appid = 730                       # CS:GO/CS2 (confirm)
contextid = 2                     

items, raw = inventory_items_for_user(steamid64, appid, contextid)
print(f"Found {len(items)} items")
for i, it in enumerate(items[:200], start=1):
    print(f"{i}. {it['name']!r} {it['market_hash_name']!r} ({it['assetid']}) - tradable={it.get('tradable')} marketable={it.get('marketable')}")

Found 48 items
1. 'SG 553 | Night Camo' 'SG 553 | Night Camo (Battle-Scarred)' (47655223284) - tradable=1 marketable=1
2. 'Revolution Case' 'Revolution Case' (47655223283) - tradable=1 marketable=1
3. 'SSG 08 | Blue Spruce' 'SSG 08 | Blue Spruce (Field-Tested)' (47501101767) - tradable=1 marketable=1
4. 'Dreams & Nightmares Case' 'Dreams & Nightmares Case' (47501101766) - tradable=1 marketable=1
5. 'StatTrak™ MAG-7 | Insomnia' 'StatTrak™ MAG-7 | Insomnia (Battle-Scarred)' (47394323285) - tradable=1 marketable=1
6. 'Sticker | Aleksib (Gold) | Copenhagen 2024' 'Sticker | Aleksib (Gold) | Copenhagen 2024' (47386508166) - tradable=1 marketable=1
7. 'StatTrak™ M4A4 | Choppa' 'StatTrak™ M4A4 | Choppa (Field-Tested)' (47381099310) - tradable=1 marketable=1
8. 'FAMAS | Meow 36' 'FAMAS | Meow 36 (Minimal Wear)' (47380823343) - tradable=1 marketable=1
9. 'Glock-18 | Block-18' 'Glock-18 | Block-18 (Field-Tested)' (47380815253) - tradable=1 marketable=1
10. '★ Kukri Knife | Night Stripe' '★ Kukri 

In [7]:
item_summaries = []#this for latter fetch on marketoverpice
seen_names = set()#this for checking duplicates market_hash_name
for i, it in enumerate(items[:200], start=1):
    name = it['market_hash_name']
    #skip if name are already added
    if not name or name in seen_names:
        continue
    seen_names.add(name)
    summary = {
        "index": i,
        "name": it['name'],
        "market_hash_name": it['market_hash_name'],
        "tradable": it.get('tradable'),
        "marketable": it.get('marketable')
    }
    item_summaries.append(summary)

print(item_summaries)

[{'index': 1, 'name': 'SG 553 | Night Camo', 'market_hash_name': 'SG 553 | Night Camo (Battle-Scarred)', 'tradable': 1, 'marketable': 1}, {'index': 2, 'name': 'Revolution Case', 'market_hash_name': 'Revolution Case', 'tradable': 1, 'marketable': 1}, {'index': 3, 'name': 'SSG 08 | Blue Spruce', 'market_hash_name': 'SSG 08 | Blue Spruce (Field-Tested)', 'tradable': 1, 'marketable': 1}, {'index': 4, 'name': 'Dreams & Nightmares Case', 'market_hash_name': 'Dreams & Nightmares Case', 'tradable': 1, 'marketable': 1}, {'index': 5, 'name': 'StatTrak™ MAG-7 | Insomnia', 'market_hash_name': 'StatTrak™ MAG-7 | Insomnia (Battle-Scarred)', 'tradable': 1, 'marketable': 1}, {'index': 6, 'name': 'Sticker | Aleksib (Gold) | Copenhagen 2024', 'market_hash_name': 'Sticker | Aleksib (Gold) | Copenhagen 2024', 'tradable': 1, 'marketable': 1}, {'index': 7, 'name': 'StatTrak™ M4A4 | Choppa', 'market_hash_name': 'StatTrak™ M4A4 | Choppa (Field-Tested)', 'tradable': 1, 'marketable': 1}, {'index': 8, 'name': 'F

In [ ]:
import browser_cookie3
import requests

# Load cookies from Chrome ()
cj = browser_cookie3.chrome(domain_name='buff.163.com')
#The only cookies info extracted is Device-ID (on none logged-in browser)
print(cj)

<CookieJar[<Cookie Device-Id=rKq84mxGnxiGibkrvb68 for buff.163.com/>]>


In [21]:
#now, use the name of items, and fetch the current market price
#Note: we do need cookies for this request.
from urllib.parse import quote
HEADERS2 = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                  "AppleWebKit/537.36 (KHTML, like Gecko) "
                  "Chrome/120.0.0.0 Safari/537.36",
    "Referer": "https://buff.163.com/",
    "Accept": "application/json, text/javascript, */*; q=0.01",
    "X-Requested-With": "XMLHttpRequest"
}
def get_market_price(market_hash_name):
    base_url = "http://buff.163.com/api/market/goods"
    url = base_url + f"?game=csgo&page_num=1&search=" +quote(market_hash_name)+"&tab=selling"
    #We need the cookies input manually, or getting the cookies on a logged on browser onto steam automatically
    #Seems like buff also using session. We can copy and extract the cookies from an logged in browser, but for automation process, its not practical.
    # cookie_str = f""

    # cookies = dict(item.split("=", 1) for item in cookie_str.split("; "))
    resp = requests.get(url,headers=HEADERS)
    #update: we wont need cookies, just change the url to http, instead of https.
    # resp = requests.get(url,headers=HEADERS2)
    
    print(f"\n>>> Request: {url}")
    print(f"Status: {resp.status_code}")
    print("Content-Type:", resp.headers.get("Content-Type"))
    print("Response snippet:", resp.text[:300])  # peek at the start
    #I'm trying to print out the response here.
    try:
        data = resp.json()
    except ValueError:
        print("❌ Response was not JSON. Probably got an HTML login page or block.")
        return None

    print("Parsed JSON keys:", list(data.keys())[:10])

    if data.get("success"):
        print("✅ Success:", data)
        return data
    else:
        print("❌ No 'success' key or request failed.")
        return None

In [ ]:
# import time
# import random
# from collections import defaultdict
# data_by_timestamp = defaultdict(dict)
# timestamp = int(time.time())
# for item in item_summaries:
#     name = item["market_hash_name"]
#     price_info = get_market_price(name)
    
#     if price_info:
#         print(price_info)
#         #data_by_timestamp[timestamp][name] = price_info
#     #     print(f"{name}: {price_info['lowest_price']} (median: {price_info['median_price']})")
#     else:
#         print(f"{name}: Price not found")
#     #adding delays for bypass steam's rate limiting
#     time.sleep(random.uniform(4.5, 6.0))


In [22]:
get_market_price("Kukri Knife | Night Stripe (Field-Tested)")


>>> Request: http://buff.163.com/api/market/goods?game=csgo&page_num=1&search=Kukri%20Knife%20%7C%20Night%20Stripe%20%28Field-Tested%29&tab=selling
Status: 200
Content-Type: application/json
Response snippet: {"code":"Login Required","error":"请先登录","extra":null}

Parsed JSON keys: ['code', 'error', 'extra']
❌ No 'success' key or request failed.
